<h2> Model training </h2>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
## BRISE UPOZORENJA
import warnings
warnings.filterwarnings('ignore')

In [3]:
## UČITAVANJE TRAIN I TEST SKUPA

X_train = pd.read_hdf('X_train_proba.h5' , key = 'X_train')
X_test = pd.read_hdf('X_test_proba.h5', key = 'X_test')
y_train = X_train['PRIJEVREMENI_RASKID'].ravel()
X_train.drop(['PRIJEVREMENI_RASKID'], axis = 1, inplace = True)

## to je normalizirani skup, spreman za treniranje

In [4]:
### s odabirom znacajki
#X_train = pd.read_hdf('X_train_ft.h5' , key = 'X_train_ft')
#X_test = pd.read_hdf('X_test_ft.h5', key = 'X_test_ft')
#y_train = X_train['PRIJEVREMENI_RASKID'].ravel()
#X_train.drop(['PRIJEVREMENI_RASKID'], axis = 1, inplace = True)


<h4> Oversampling </h4>

Po volji napravimo i ovo.

In [ ]:
from sklearn.utils import resample
from sklearn.model_selection import train_test_split

# concatenate our training data back together
X = pd.concat([X_train, y_train], axis=1)

# separate minority and majority classes
ne_raskidi = X[X['PRIJEVREMENI_RASKID']=='N']
raskidi = X[X['PRIJEVREMENI_RASKID']=='Y']

# upsample minority
raskidi_upsampled = resample(raskidi,
                          replace=True, # sample with replacement
                          n_samples=0.75*len(ne_raskidi), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([neraskidi, raskidi_upsampled])

# check new class counts
upsampled['PRIJEVREMENI_RASKID'].value_counts()

<h3> XGBoost </h3>

In [5]:
#!pip install xgboost

In [4]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [7]:
# https://www.kaggle.com/babatee/intro-xgboost-classification
#model1 = xgb.XGBClassifier()
#model2 = xgb.XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5)

In [8]:
#train_model1 = model1.fit(X_train, y_train)
#train_model2 = model2.fit(X_train, y_train)

In [ ]:
#from sklearn.metrics import classification_report

#pred1 = train_model1.predict(X_test)
#pred2 = train_model2.predict(X_test)

In [ ]:
# ISPIS RJESENJA

#y_test = pred2
#y_test = np.where(y_test==0,'N','Y')
#student = pd.read_excel('eval_dataset_nan.xlsx')
#student['PRIJEVREMENI_RASKID'] = y_test
#student.to_csv('student.csv')

<h4> Hyperparameter tuning </h4>

In [ ]:
#model1 = xgb.XGBClassifier( learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)

In [ ]:
#train_model1 = model1.fit(X_train, y_train)
#pred3 = train_model1.predict(X_test)
# print("Accuracy for model 3: %.2f" % (accuracy_score(y_test, pred3) * 100))

In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
rez = {'Accuracy': make_scorer(accuracy_score)}


param_test = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
gsearch = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=200, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test, scoring=rez, n_jobs=4,iid=False, refit = 'Accuracy' ,cv=5)

train_model4 = gsearch.fit(X_train, y_train)
pred4 = train_model4.predict(X_test)
#print("Accuracy for model 4: %.2f" % (accuracy_score(y_test, pred4) * 100))

NameError: name 'xgb' is not defined

In [ ]:
gsearch.best_params_ # pokaze koji je najbolji parametar odabran

In [ ]:
gsearch.best_score_

In [29]:
##ISPIS RJESENJA
y_test = pred4
y_test = np.where(y_test==0,'N','Y')
student = pd.read_excel('eval_dataset_nan.xlsx')
student['PRIJEVREMENI_RASKID'] = y_test
student.to_csv('student.csv')

In [ ]:
param_test2b = {
 'min_child_weight':[6,8,10,12]
}
gsearch2b = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=200, max_depth=4,
 min_child_weight=5,  gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2b, scoring=rez, refit = 'Accuracy',n_jobs=4,iid=False, cv=5)

train_model5 = gsearch2b.fit(X_train, y_train)
pred5 = train_model5.predict(X_test)
#print("Accuracy for model 5: %.2f" % (accuracy_score(y_test, pred5) * 100))

In [ ]:
gsearch2b.best_params_

In [ ]:
gsearch2b.best_score_

In [14]:
##ISPIS RJESENJA
y_test = pred5
y_test = np.where(y_test==0,'N','Y')
student['PRIJEVREMENI_RASKID'] = y_test
student.to_csv('student.csv')

In [31]:
#Tune Gamma
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=200, max_depth=4,
 min_child_weight=6, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring=rez, refit = 'Accuracy',n_jobs=4,iid=False, cv=5)

train_model6 = gsearch3.fit(X_train, y_train)
pred6 = train_model6.predict(X_test)
#print("Accuracy for model 6: %.2f" % (accuracy_score(y_test, pred6) * 100))

KeyboardInterrupt: 

In [23]:
gsearch3.best_params_

{'gamma': 0.0}

In [ ]:
xgb2 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=500,
 max_depth=4,
 min_child_weight=4,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

train_model7 = xgb2.fit(X_train, y_train)
pred7 = train_model7.predict(X_test)
#print("Accuracy for model 7: %.2f" % (accuracy_score(y_test, pred7) * 100))

In [ ]:
##ISPIS RJESENJA - pokazao se najbolji dosad
#y_test = pred7
#y_test = np.where(y_test==0,'N','Y')
#student = pd.read_excel('eval_dataset_nan.xlsx')
#student['PRIJEVREMENI_RASKID'] = y_test
#student.to_csv('student.csv')

Drugi skup

In [ ]:
## UČITAVANJE TRAIN I TEST SKUPA

X_train = pd.read_hdf('X_train_ft1.h5' , key = 'X_train_ft2')
X_test = pd.read_hdf('X_test_ft1.h5', key = 'X_test_ft1')
y_train = X_train['PRIJEVREMENI_RASKID'].ravel()
X_train.drop(['PRIJEVREMENI_RASKID'], axis = 1, inplace = True)

## to je normalizirani skup, spreman za treniranje



In [ ]:
from sklearn.model_selection import GridSearchCV

param_test = {
 'max_depth':[4,5,6],
 'min_child_weight':[4,5,6]
}
gsearch_2 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=500, max_depth=5,
 min_child_weight=2, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test, scoring=rez, refit = 'Accuracy',n_jobs=4,iid=False, cv=5)

train_model4_2 = gsearch_2.fit(X_train, y_train)
pred4_2 = train_model4_2.predict(X_test)
#print("Accuracy for model 4: %.2f" % (accuracy_score(y_test, pred4) * 100))

print(gsearch_2.best_params_)

In [ ]:
param_test2b = {
 'min_child_weight':[6,8,10,12]
}
gsearch2b_2 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate=0.1, n_estimators=500, max_depth=4,
 min_child_weight=4, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test2b, scoring='roc_auc',n_jobs=4,iid=False, cv=5)

train_model5_2 = gsearch2b_2.fit(X_train, y_train)
pred5_2 = train_model5_2.predict(X_test)
#print("Accuracy for model 5: %.2f" % (accuracy_score(y_test, pred5) * 100))

In [ ]:
#Tune Gamma
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3_2 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=500, max_depth=4,
 min_child_weight=4, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1,seed=27), 
 param_grid = param_test3, scoring=rez, refit ='Accuracy',n_jobs=4,iid=False, cv=5)

train_model6_2 = gsearch3_2.fit(X_train, y_train)
pred6_2 = train_model6_2.predict(X_test)
#print("Accuracy for model 6: %.2f" % (accuracy_score(y_test, pred6) * 100))

In [ ]:
xgb2 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=500,
 max_depth=4,
 min_child_weight=4,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)

train_model7_2 = xgb2_2.fit(X_train, y_train)
pred7_2 = train_model7_2.predict(X_test)
#print("Accuracy for model 7: %.2f" % (accuracy_score(y_test, pred7) * 100))

<h2> Random Forest </h2>

In [ ]:
## Memory Error

#from sklearn.ensemble import RandomForestClassifier

#Create a Gaussian Classifier
#clf=RandomForestClassifier(n_estimators=1000)

#Train the model using the training sets y_pred=clf.predict(X_test)
#clf.fit(X_train,y_train)

#y_pred=clf.predict(X_test)